In [4]:
from pydataset import data
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [7]:
# Get the data
titanic = data('titanic')

# pick a random sample
titanic.sample(5)

,class,age,sex,survived
963,3rd class,adults,man,no
1277,3rd class,child,man,no
812,3rd class,adults,man,no
118,1st class,adults,man,no
743,3rd class,adults,man,no


# Featuring engineering

In [33]:
# One Hot Enconding
titanic = pd.get_dummies(titanic, drop_first=True).astype(int)

display(titanic.sample(5))

,class_2nd class,class_3rd class,age_child,sex_women,survived_yes
205,0,0,0,1,1
603,1,0,1,1,1
834,0,1,0,0,0
475,1,0,0,0,0
801,0,1,0,0,0


# Split the data

In [21]:
X = np.array(titanic.drop('survived_yes', axis=1))
y = np.array(titanic['survived_yes'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model

In [24]:
# train
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

# Predictions

In [34]:
# Predictions
print('classe-1 child girl:  ', model.predict(np.array([[0, 0, 1, 1]]))[0])
print('class-3 adult-age male:  ', model.predict(np.array([[0, 0, 1, 1]]))[0])

classe-1 child girl:   1
class-3 adult-age male:   1


# Scoring

In [37]:
# scoring the model
model.score(X_test, y_test)

0.7954545454545454

In [41]:
predictions = (model.predict(X_test) > .5).astype(int)
np.sum(predictions == y_test) / len(y_test)


0.7954545454545454